# Practice KNN,LR,SVM

# Contnet <a id='Jump'></a>

##   1.[Import and reading the Data Section](#imports)
---
##   2.[Exploration](#EDA)<br>
---
##   3.[DataPreprocess](#preprocess)
###  <br>3.1.[Clean NA values](#NA)

###  <br>3.2.[Scalling](#scaling)
###  <br>3.3.[Clean Duplication](#Duplication)<br>
---
## 4.[Modeling](#Models)<br>
### 4.1.[KNN](#KNN)
### 4.2.[Logistic  Regression](#lr)
### 4.3.[SVM](#svm)

## imports
<a id='imports'></a>
##### [Contnet](#Jump)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore')
from IPython.display import display, HTML
#preprocess
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE

#model selection
from sklearn.model_selection import train_test_split , GridSearchCV, cross_val_score, cross_val_predict

#models.
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

#validation 
from sklearn.metrics import classification_report , precision_score, recall_score, f1_score, accuracy_score, roc_auc_score, roc_curve

In [ ]:
#at kaggels 
df = pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')
#on my PC 
# df = pd.read_csv('diabetes.csv')
print(df.shape)
df.head(1)

### Helps

In [ ]:
from IPython.display import display_html
from itertools import chain,cycle
def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2>{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

## Exploration
<a id='EDA'></a>
##### [Contnet](#Jump)

In [ ]:
df.info()

- all int or float that makes things simpler 
- also there is no na here

In [ ]:
df[df.duplicated()]

- no duplicates in the data

In [ ]:
 df.nunique(axis=0).sort_values().to_frame()

In [ ]:
df.describe()

In [ ]:
df.columns

the columns info 

    Pregnancies:	"Number of times pregnant"
    Glucose:	Plasma glucose concentration a 2 hours in an oral glucose tolerance test
    BloodPressure:	"Diastolic blood pressure (mm Hg)"
    SkinThickness:	Triceps skin fold thickness (mm)
    Insulin:	2-Hour serum insulin (mu U/ml)
    BMI:	Body mass index (weight in kg/(height in m)^2)
    DiabetesPedigreeFunction:	Diabetes pedigree function
    Age:	Age (years)
    Outcome:	Class variable (0 or 1) 268 of 768 are 1, the others are 0

[source](https://machinelearningmastery.com/case-study-predicting-the-onset-of-diabetes-within-five-years-part-1-of-3/)
A particularly interesting attribute used in the study was the **Diabetes Pedigree Function**, pedi. It provided some data on diabetes mellitus history in relatives and the genetic relationship of those relatives to the patient. This measure of genetic influence gave us an idea of the hereditary risk one might have with the onset of diabetes mellitus. Based on observations in the proceeding section, it is unclear how well this function predicts the onset of diabetes.

[anothersource](https://rpubs.com/ikodesh/53189)
According to http://www.personal.kent.edu/~mshanker/personal/Zip_files/sar_2000.pdf, the diabetes pedigree function provides “a synthesis of the diabetes mellitus history in relatives and the genetic relationship of those relatives to the subject.” It utilizes information from a person’s family history to predict how diabetes will affect that individual. According to http://www.rci.rutgers.edu/~cabrera/587/pima.pdf, “many Pima Indians have diabetes”. I was intrigued at why this might be and what variables may influence this. To test the relationship between BMI and the number of times the women were pregnant, I will compare the two variables visually.

columns cant be zeros ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin','BMI', 'Age' , 'DiabetesPedigreeFunction']

In [ ]:
 #this rename for much easiar read 
df.rename(columns={'DiabetesPedigreeFunction': 'pedi'},inplace=True)

In [ ]:
Cant_zeros = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin','BMI', 'Age' ,'pedi']
df[Cant_zeros] = df[Cant_zeros].replace(0, np.nan)

In [ ]:
df[Cant_zeros].describe()

In [ ]:
df.describe().T

In [ ]:
 df.hist(figsize = (20,20));

In [ ]:
na = df.isna().sum().sort_values(ascending =False)
NA = na[na>0].to_frame()
NA.columns = ['Nulls']

In [ ]:
explorenulls ='display_side_by_side(NA,df[NA.index.to_list()].nunique(axis=0).to_frame().set_axis(["uniqu"], axis=1))'

In [ ]:
exec(explorenulls)
df.Insulin.hist(figsize = (2,2));
display(df[~df.Insulin.isna()])
display(df[['Insulin']].describe())
sns.pairplot(df, y_vars="Insulin", x_vars=df.columns.values);

i will fill inslun with median as there is no realy connetction betweet it and other var to help me fill it and the var of the data is really big i cant just fill it with the mean that might ruin the data and this column need some expert to help me fill it 
<a id='deal_with_Insulin'></a>

In [ ]:
exec(explorenulls)
df.SkinThickness.hist(figsize = (2,2));
display(df[df.SkinThickness.isna()])
sns.pairplot(df, y_vars="SkinThickness", x_vars=df.columns.values);
display(df[['SkinThickness']].describe())

Same as The insulin fill it with median 
<a id='deal_with_SkinThickness'></a>

In [ ]:
exec(explorenulls)
df.BloodPressure.hist(figsize = (2,2));
display(df[df.BloodPressure.isna()])
sns.pairplot(df, y_vars="BloodPressure", x_vars=df.columns.values);
display(df[['BloodPressure']].describe())

i can fill it with mean as the distrpution wont go bad 
<a id='deal_with_BloodPressure'></a>

In [ ]:
exec(explorenulls)
df.BMI.hist(figsize = (2,2));
display(df[df.BMI.isna()])
sns.pairplot(df, y_vars="BMI", x_vars=df.columns.values);
display(df[['BMI']].describe())

i can use skinthickness to help me fill the data here create small regressing model 
<a id='deal_with_BMI'></a>

In [ ]:
exec(explorenulls)
df.Glucose.hist(figsize = (2,2));
display(df[df.Glucose.isna()])
sns.pairplot(df, y_vars="Glucose", x_vars=df.columns.values);
display(df[['Glucose']].describe())

i will fill it with mean 
<a id='deal_with_Glucose'></a>


##  3.[DataPreprocess](#preprocess)
<a id='preprocess'></a>
##### [Contnet](#Jump)

#### split the data set before continue 

In [ ]:
TrainDF, TestDF = train_test_split(df, test_size=0.2, random_state=9110)

### 3.1.[Clean NA values](#NA)
<a id='NA'></a>
##### [Contnet](#Jump)

[Insulin](#deal_with_Insulin)

In [ ]:
col = 'Insulin' 
TrainDF[col].fillna(TrainDF[col].median(), inplace = True)
TestDF[col].fillna(TrainDF[col].median(), inplace = True)

[deal with SkinThickness](#deal_with_SkinThickness)

In [ ]:
col = 'SkinThickness' 
TrainDF[col].fillna(TrainDF[col].median(), inplace = True)
TestDF[col].fillna(TrainDF[col].median(), inplace = True)

[deal with BloodPressure](#deal_with_BloodPressure)

In [ ]:
col = 'BloodPressure' 
TrainDF[col].fillna(TrainDF[col].mean(), inplace = True)
TestDF[col].fillna(TrainDF[col].mean(), inplace = True)

[deal with BMI](#deal_with_BMI)

In [ ]:
from sklearn.linear_model import LinearRegression
data = TrainDF[['SkinThickness','BMI']][TrainDF.BMI.notna()] # load data set
X = data.iloc[:, 0].values.reshape(-1, 1)  # values converts it into a numpy array
Y = data.iloc[:, 1].values.reshape(-1, 1)  # -1 means that calculate the dimension of rows, but have 1 column
linear_regressor = LinearRegression()  # create object for the class
linear_regressor.fit(X, Y)  # perform linear regression
TrainDF.loc[TrainDF.BMI.isna(),'BMI']= linear_regressor.predict(TrainDF['SkinThickness'][TrainDF.BMI.isna()].values.reshape(-1, 1) )
TestDF.loc[TestDF.BMI.isna(),'BMI']= linear_regressor.predict(TestDF['SkinThickness'][TestDF.BMI.isna()].values.reshape(-1, 1) ) 

[deal with Glucose](#deal_with_Glucose)

In [ ]:
col = 'Glucose' 
TrainDF[col].fillna(TrainDF[col].mean(), inplace = True)
TestDF[col].fillna(TrainDF[col].mean(), inplace = True)

In [ ]:
TrainDF.isna().sum()

In [ ]:
TestDF.isna().sum()

#### 3.2.[Data Scaling ](#scaling) 
<a id='scaling'></a>
##### [Contnet](#Jump)

In [ ]:
features = df.columns.to_list()
features.pop(-1)
features

In [ ]:
scaler = StandardScaler()
TrainDF[features] = scaler.fit_transform(TrainDF[features])
TestDF[features] = scaler.transform(TestDF[features])

### 3.3.[Clean Duplication](#Duplication)
<a id='Duplication'></a>
##### [Contnet](#Jump)

Wont do that i want to fix imlance data 

In [ ]:
X_train,y_train = TrainDF.drop(['Outcome'],axis = 1),TrainDF[['Outcome']]
X_test,y_test = TestDF.drop(['Outcome'],axis = 1),TestDF[['Outcome']]

### 3.4.Imb


In [ ]:
# transform the dataset
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_train_S,y_train_S = oversample.fit_resample(X_train,y_train)

## 4.[Modeling](#Models)
### 4.1.[KNN](#KNN)
### 4.2.[Logistic  Regression](#lr)
### 4.3.[SVM](#svm)
<a id='Models'></a>
##### [Contnet](#Jump)


### 4.1.[KNN](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)
<a id='KNN'></a>
##### [Contnet](#Jump)

In [ ]:
knn = KNeighborsClassifier()
param_grid = {
    'n_neighbors': [*range(1,20)],
}
Grid_search = GridSearchCV(knn, param_grid=param_grid, cv=5, scoring='roc_auc',n_jobs=-1)

Grid_search.fit(X_train , y_train)
Grid_search.best_estimator_
finalknn  = Grid_search.best_estimator_
finalknn.fit(X_train,y_train )
y_model = finalknn.predict(X_test)
print('roc_auc_score : ', roc_auc_score(y_test, y_model))
print('accuracy_score : ', accuracy_score(y_test, y_model))
print('recall_score : ', recall_score(y_test, y_model))
print('precision : ', precision_score(y_test, y_model))
print('f1_score : ', f1_score(y_test, y_model))
print(classification_report(y_test, y_model))
finalknn

In [ ]:
knn = KNeighborsClassifier()
param_grid = {
    'n_neighbors': [*range(1,20)],
}
Grid_search = GridSearchCV(knn, param_grid=param_grid, cv=5, scoring='roc_auc',n_jobs=-1)

Grid_search.fit(X_train_S,y_train_S)
Grid_search.best_estimator_
finalknn  = Grid_search.best_estimator_
finalknn.fit(X_train_S,y_train_S )
y_model = finalknn.predict(X_test)
print("With Smote")
print('roc_auc_score : ', roc_auc_score(y_test, y_model))
print('accuracy_score : ', accuracy_score(y_test, y_model))
print('recall_score : ', recall_score(y_test, y_model))
print('precision : ', precision_score(y_test, y_model))
print('f1_score : ', f1_score(y_test, y_model))
print(classification_report(y_test, y_model))
finalknn

### 4.2.Logistic Regression
<a id='lr'></a>
##### [Contnet](#Jump)

(penalty='l2', *, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)

Cfloat, default=1.0
Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.

In [ ]:
lr = LogisticRegression(random_state=9110,n_jobs=-1,max_iter=100)
param_grid = {
    'C': [0.0001, 0.001, 0.01,0.1,1,5],
    'solver': ['newton-cg', 'lbfgs', 'liblinear'],
}

Grid_search = GridSearchCV(lr, param_grid=param_grid, cv=5, scoring='roc_auc',n_jobs=-1)

Grid_search.fit(X_train , y_train)
Grid_search.best_estimator_
finallr = Grid_search.best_estimator_
finallr.fit(X_train,y_train )
y_model = finallr.predict(X_test)
print('roc_auc_score : ', roc_auc_score(y_test, y_model))
print('accuracy_score : ', accuracy_score(y_test, y_model))
print('recall_score : ', recall_score(y_test, y_model))
print('precision : ', precision_score(y_test, y_model))
print('f1_score : ', f1_score(y_test, y_model))
print(classification_report(y_test, y_model))
finallr

In [ ]:
lr = LogisticRegression(random_state=9110,n_jobs=-1,max_iter=100)
param_grid = {
    'C': [0.0001, 0.001, 0.01,0.1,1,5],
    'solver': ['newton-cg', 'lbfgs', 'liblinear'],
}

Grid_search = GridSearchCV(lr, param_grid=param_grid, cv=5, scoring='roc_auc',n_jobs=-1)

Grid_search.fit(X_train_S,y_train_S)
Grid_search.best_estimator_
finallr = Grid_search.best_estimator_
finallr.fit(X_train_S,y_train_S )
y_model = finallr.predict(X_test)
print("With Smote")
print('roc_auc_score : ', roc_auc_score(y_test, y_model))
print('accuracy_score : ', accuracy_score(y_test, y_model))
print('recall_score : ', recall_score(y_test, y_model))
print('precision : ', precision_score(y_test, y_model))
print('f1_score : ', f1_score(y_test, y_model))
print(classification_report(y_test, y_model))
finallr

### 4.3[SVM](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)
<a id='svm'></a>
##### [Contnet](#Jump)

C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=- 1, decision_function_shape='ovr', break_ties=False, random_state=None)[source]¶

kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}, default=’rbf

In [ ]:
svm = SVC(random_state = 9110)
param_grid = {
    'C': [0.0001, 0.001, 0.01,0.1,1,5],
    'kernel': ['linear', 'poly', 'rbf','sigmoid'],
    'degree': [2,3,4,5],
    'shrinking': [True,False]
}

Grid_search = GridSearchCV(svm, param_grid=param_grid, cv=5, scoring='roc_auc',n_jobs=-1)

Grid_search.fit(X_train , y_train)
Grid_search.best_estimator_
finalsvm = Grid_search.best_estimator_
finalsvm.fit(X_train,y_train )
y_model = finalsvm.predict(X_test)
print('roc_auc_score : ', roc_auc_score(y_test, y_model))
print('accuracy_score : ', accuracy_score(y_test, y_model))
print('recall_score : ', recall_score(y_test, y_model))
print('precision : ', precision_score(y_test, y_model))
print('f1_score : ', f1_score(y_test, y_model))
print(classification_report(y_test, y_model))
finalsvm

In [ ]:
svm = SVC(random_state = 9110)
param_grid = {
    'C': [0.0001, 0.001, 0.01,0.1,1,5],
    'kernel': ['linear', 'poly', 'rbf','sigmoid'],
    'degree': [2,3,4,5],
    'shrinking': [True,False]
}

Grid_search = GridSearchCV(svm, param_grid=param_grid, cv=5, scoring='roc_auc',n_jobs=-1)

Grid_search.fit(X_train_S,y_train_S)
Grid_search.best_estimator_
finalsvm = Grid_search.best_estimator_
finalsvm.fit(X_train_S,y_train_S )
y_model = finalsvm.predict(X_test)
print("With Smote")
print('roc_auc_score : ', roc_auc_score(y_test, y_model))
print('accuracy_score : ', accuracy_score(y_test, y_model))
print('recall_score : ', recall_score(y_test, y_model))
print('precision : ', precision_score(y_test, y_model))
print('f1_score : ', f1_score(y_test, y_model))
print(classification_report(y_test, y_model))
finalsvm